In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate

In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
# Initialize models
orchestrator = ChatOpenAI(model="gpt-4o-mini", temperature=0)
plan_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
draft_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
critic_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [10]:
search_tool = DuckDuckGoSearchRun()
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectordb = Chroma(persist_directory="./vectorstore", embedding_function=embeddings)

In [11]:
def route_query(query):
    """Classify query as general or research."""
    prompt = f"Classify this query as 'general' or 'research': {query}"
    result = orchestrator.invoke(prompt)
    return result.content.strip().lower()

def general_answer(query):
    return plan_llm.invoke(query).content

def analyzer_collect(query):
    """Retrieve context using web search + Chroma."""
    web_results = search_tool.run(query)
    vectordb.add_texts([web_results])
    retriever = vectordb.as_retriever(search_kwargs={"k": 3})
    context_docs = retriever.invoke(query)
    return "\n\n".join([d.page_content for d in context_docs])

def plan_writer(query, context):
    plan_prompt = ChatPromptTemplate.from_template("""
    Based on the user's goal:
    {query}

    And this context:
    {context}

    Write a clear, step-by-step plan.
    """)
    return plan_llm.invoke(plan_prompt.format(query=query, context=context)).content

def draft_writer(plan):
    return draft_llm.invoke(f"Write a structured essay based on this plan:\n{plan}").content

def critic_agent(draft):
    return critic_llm.invoke(f"Critique this draft and suggest improvements:\n{draft}").content

def final_drafter(text):
    final_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
    return final_llm.invoke(f"Polish and finalize this essay:\n{text}").content

def orchestrate_agent(query):
    intent = route_query(query)
    if intent == "general":
        return general_answer(query)
    else:
        context = analyzer_collect(query)
        plan = plan_writer(query, context)
        draft = draft_writer(plan)
        critique = critic_agent(draft)
        print("🧠 Critique:", critique)
        final = final_drafter(draft)
        vectordb.add_texts([final])
        return final